In [1]:
from sklearn import linear_model
import numpy as np
from sklearn.model_selection import cross_validate
import json
from itertools import combinations
from sklearn.metrics import mean_squared_error
from math import sqrt

In [4]:
# read data
training_x = list()
with open ("../training_X_with_text_features.json",'r') as data:
    for d in data:
        training_x = json.loads(d)
        
# read data
training_y = list()
with open ("../training_Y_with_text_features.json",'r') as data:
    for d in data:
        training_y = json.loads(d)
        
training_x = np.array(training_x)
training_y = np.array(training_y)

print(training_x.shape)
print(training_y.shape)

# read data
testing_x = list()
with open ("../testing_X_with_text_features.json",'r') as data:
    for d in data:
        testing_x = json.loads(d)
        
# read data
testing_y = list()
with open ("../testing_Y_with_text_features.json",'r') as data:
    for d in data:
        testing_y = json.loads(d)
        
testing_x = np.array(testing_x)
testing_y = np.array(testing_y)

print(testing_x.shape)
print(testing_y.shape)

(1237673, 20)
(1237673,)
(280588, 20)
(280588,)


In [5]:
part_training_x = training_x[:round(0.25*1237673),:]
part_training_y = training_y[:round(0.25*1237673)]
print(part_training_x.shape)
print(part_training_y.shape)

(309418, 20)
(309418,)


In [35]:
# list all combinations of 20 features
index = [*range(0,20)]
index_combs = []

for i in range(0,len(index)+1):
    index_combs.append(list((combinations(index,i))))
    
# parse the list
index_comb_parsed = []
for index_comb in index_combs:
    for i in index_comb:
        i = list(i)
        if len(i) != 0:
            index_comb_parsed.append(i)

In [36]:
# number of all combinations
len(index_comb_parsed)

1048575

In [44]:
def ridge_cv():
    
    # use cross validation to fit the model, and save the results of a dictionary of <rmse, indexes>
    
    results_dict = dict()
    count = 1

    for t in index_comb_parsed:

        # you can change to another model
        ridge = linear_model.Ridge(alpha=0.5)

        part_features = part_training_x[:,t]
        cv_results = cross_validate(ridge, part_features, part_training_y, cv=3, scoring=('neg_mean_squared_error'))

        results = -cv_results['test_score']
        results_dict[round(np.mean(results),5)] = t

        count += 1

        if count % 1000 == 0:
            print(count)
            
    return results_dict

In [49]:
# results_dict = ridge_cv()

In [50]:
# cross validation results: top 10 with lowest RMSE

In [43]:
sorted(results_dict.items(),key = lambda x:x[0], reverse = False)[:10]

[(1.14507, [0, 1, 2, 6, 7, 9, 10, 11, 12, 16, 18, 19]),
 (1.14509, [0, 1, 2, 6, 9, 10, 11, 12, 13, 16, 18, 19]),
 (1.1451, [1, 2, 4, 6, 7, 9, 10, 11, 12, 16, 18, 19]),
 (1.14511, [0, 1, 2, 6, 9, 10, 11, 12, 16, 17, 18, 19]),
 (1.14512, [1, 2, 6, 7, 9, 10, 11, 12, 15, 16, 18, 19]),
 (1.14513, [1, 2, 4, 6, 9, 10, 11, 12, 15, 16, 18, 19]),
 (1.14514, [1, 2, 6, 7, 9, 10, 11, 12, 16, 17, 18, 19]),
 (1.14515, [1, 2, 6, 9, 10, 11, 12, 13, 15, 16, 18, 19]),
 (1.14516, [1, 2, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19]),
 (1.14517, [1, 2, 6, 9, 10, 11, 12, 15, 16, 17, 18, 19])]

In [51]:
# we will be using the feature set that gives the optimal rmse
# the features are [0, 1, 2, 6, 7, 9, 10, 11, 12, 16, 18, 19]
# we re-fit the model with these features, and use all the data

In [6]:
useful_feature_index = [0, 1, 2, 6, 7, 9, 10, 11, 12, 16, 18, 19]

training_features = training_x[:,useful_feature_index]
testing_features = testing_x[:,useful_feature_index]

In [7]:
ridge = linear_model.Ridge(alpha=0.5)
ridge.fit(training_features,training_y)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [8]:
predicted_ratings = ridge.predict(testing_features)

In [9]:
predicted_ratings

array([3.90719468, 3.36122287, 3.53238204, ..., 2.27614801, 3.29654558,
       3.30178419])

In [10]:
rmse = sqrt(mean_squared_error(predicted_ratings, testing_y))

In [11]:
rmse

1.3336532751405283